# ID2214/FID3214 Assignment 3 Group no. [enter]
### Project members: 
[Enter Name, email]
[Enter Name, email]
[Enter Name, email]
[Enter Name, email]

### Declaration
By submitting this solution, it is hereby declared that all individuals listed above have contributed to the solution, either with code that appear in the final solution below, or with code that has been evaluated and compared to the final solution, but for some reason has been excluded. It is also declared that all project members fully understand all parts of the final solution and can explain it upon request.

It is furthermore declared that the code below is a contribution by the project members only, and specifically that no part of the solution has been copied from any other source (except for lecture slides at the course ID2214/FID3214) and no part of the solution has been provided by someone not listed as project member above.

It is furthermore declared that it has been understood that no other library/package than the Python 3 standard library, NumPy, pandas, time and sklearn.tree, may be used in the solution for this assignment.

### Instructions
All parts of the assignment starting with number 1 below are mandatory. Satisfactory solutions
will give 1 point (in total). If they in addition are good (all parts work more or less 
as they should), completed on time (submitted before the deadline in Canvas) and according
to the instructions, together with satisfactory solutions of all parts of the assignment starting 
with number 2 below, then the assignment will receive 2 points (in total).

Note that you do not have to develop the code directly within the notebook
but may instead copy the comments and test cases to a more convenient development environment
and when everything works as expected, you may paste your functions into this
notebook, do a final testing (all cells should succeed) and submit the whole notebook 
(a single file) in Canvas (do not forget to fill in your group number and names above).

## Load NumPy, pandas, time and DecisionTreeClassifier from sklearn.tree

In [1]:
import numpy as np
import pandas as pd
import time
import sklearn
from sklearn.tree import DecisionTreeClassifier

In [2]:
from platform import python_version

print(f"Python version: {python_version()}")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"sklearn version: {sklearn.__version__}")

Python version: 3.10.11
NumPy version: 1.23.4
Pandas version: 2.0.3
sklearn version: 1.1.1


## Reused functions from Assignment 1

In [3]:
# Copy and paste functions from Assignment 1 here that you need for this assignment

# COLUMN FILTER

def create_column_filter(df):

    # Step 1. Copy the input dataframe.
    copy_df = df.copy()

    # Step 2. Initialise an empty list so to store the names of the columns witch will be kept.
    column_filter = []

    # Step 3. Iterate through all the columns in the dataframe.
    for column in copy_df.columns:

        # Step 4. Check if the column name is CLASS or ID so do not drop them.
        if column == "CLASS" or column == "ID":
            column_filter.append(column)
        else:

            # Step 5. Drop missing values and get unique values.
            unique_values = copy_df[column].dropna().unique()
            
            # Step 6. Check if the number of unique (non-missing) values is more than one, keep the column
            if (len(unique_values) > 1):
                column_filter.append(column)
            else:
                # Step 7. Drop the column from the dataframe copy.
                copy_df.drop(columns=[column], inplace=True)
        
        
    # Return the modified dataframe and the list of remaining columns
    return copy_df, column_filter

def apply_column_filter(df, column_filter):

    # Step 1. Copy the initial dataframe.
    copy_df = df.copy()

    # Step 2. Iterate through all the columns of the copied dataframe.
    for column in copy_df.columns:

        # Step 3. Check the column name if it is in the list of the filtered columns.  
        if column not in column_filter:
            # Step 4. If the column name is not in the filtered columns then it will be dropped.
            copy_df.drop(columns=[column], inplace=True)

    # Step 5. Return the modified dataframe with only the columns witch are in the filtered columns list
    return copy_df

# IMPUTATION

def create_imputation(df):

    # Step 1: Copy the input dataframe
    copy_df = df.copy()

    # Strep 2: Initialize a dictionary to store imputation values
    imputation = {}

    # Step 3: Iterate trough all the columns in the dataframe
    for column in copy_df.columns:

        # Step 4: Skip the "CLASS" and "ID" columns
        if column not in ["CLASS", "ID"]:

            # Step 5: Handle numeric columns (float and int types)
            if np.issubdtype(copy_df[column].dtype, np.number): # = if copy_df[column].dtype in ["float64", "int64"]
                # Calculate the mean value of the column
                mean_value = copy_df[column].mean()
                # Check if all the values of the column are missing and replace mean with 0
                if pd.isnull(mean_value):
                    mean_value = 0
                # Replace the missing values with the mean value
                copy_df[column].fillna(mean_value, inplace=True)
                # Add the mean value to the imputation dictionary
                imputation[column] = mean_value

            # Step 6: Handle the object and the category columns
            elif copy_df[column].dtype in ["object", "category"]:
                # Calculate the mode of the column (Find the most frequent value appear in the column)
                mode_value = copy_df[column].mode().iloc[0] if not copy_df[column].mode().empty else ""
                # If all values are missing and column is category then replace the mode value with the first category.
                # If no categories it will raise an error.
                if copy_df[column].dtype.name == "category" and mode_value == "":               
                    mode_value = copy_df[column].cat.categories[0]
                # Replace missing values with the mode value
                copy_df[column].fillna(mode_value, inplace=True)
                # Add the mode value to the imputation dictionary
                imputation[column] = mode_value

    return copy_df, imputation

def apply_imputation(df, imputation):
    # Step 1: Copy the input dataframe
    copy_df = df.copy()
    # Step 2: Iterate through the imputation dictionary and take the items of it
    for column_name, value in imputation.items():
        # Fill the empty values of the copied dataframe according the dictionary map.
        copy_df[column_name].fillna(value, inplace=True)

    return copy_df

# ONE-HOT ENCODING

def create_one_hot(df: pd.DataFrame) -> tuple[pd.DataFrame, dict]:

    # Step 1: Copy the input dataframe to avoid changes to the initial dataframe
    copy_df = df.copy()
    # Step 2: Initialize a dictionary to stor one-hot information
    one_hot = {}

    # Step 3: Iterate through the columns of the dataframe and apply the one-hot encoding to categorical columns
    for column in copy_df.columns:
        if column not in ["CLASS", "ID"] and copy_df[column].dtype in ["category", "object"]:

            # Get unique categories for the columns
            unique_column_category = df[column].unique()
            # Store the unique categories into dictionary
            one_hot[column] = unique_column_category
            # Generate one-hot encoding
            for category in sorted(unique_column_category):
                # Create a new column for each category
                new_column_category_name = f"{column}_{category}"
                copy_df[new_column_category_name] = (df[column] == category).astype(float)

            # Remove the original column
            copy_df.drop(column, axis=1, inplace=True)

    return copy_df, one_hot

def apply_one_hot(df: pd.DataFrame, one_hot: dict) -> pd.DataFrame:

    # Step 1: Copy the input dataframe to avoid changes to the initial dataframe
    copy_df = df.copy()

    # Step 2: Iterate through the one-hot dictionary so to apply the one-hot encoding
    for column, categories in one_hot.items():
        if column in df.columns:
            for category in categories:
                new_column_category_name = f"{column}_{category}"
                # Create a new column an we add 0 as the default value
                copy_df[new_column_category_name] = 0
                # We add 1 if the category matches 
                copy_df.loc[df[column] == category, new_column_category_name] = 1
            # Remove the original value
            copy_df.drop(column, axis=1, inplace=True)

    return copy_df

# ACCURACY

def accuracy(df: pd.DataFrame, correctlabels: list):

    if len(df) != len(correctlabels):
        raise ValueError("The number of rows in the DataFrame must equal the number of correct labels")
    
    # Step 2: Find the label with the highest probability in each row
    predict_labels = df.idxmax(axis=1)

    # Step 3: Compare with the correct labels
    matches = predict_labels == correctlabels

    # Step 4: Calculate accuracy 
    accuracy = matches.sum() / len(correctlabels)

    return accuracy

# AUC

def auc(df, correctlabels):
    # Helper function to calculate binary AUC for a single class
    def binary_auc(class_name):
        scores = df[class_name].values
        true_positives = [int(label == class_name) for label in correctlabels]
        false_positives = [int(label != class_name) for label in correctlabels]

        # Create triples of (score, tp, fp)
        triples = [(score, tp, fp) for score, tp, fp in zip(scores, true_positives, false_positives)]

        # Sort the triples based on scores in reverse order
        triples.sort(key=lambda x: x[0], reverse=True)

        # Calculate AUC using the lecture's algorithm
        auc = 0
        cov_tp = 0
        tot_tp = sum(true_positives)
        tot_fp = sum(false_positives)

        for _, tp, fp in triples:
            if fp == 0:
                cov_tp += tp
            elif tp == 0:
                auc += (cov_tp / tot_tp) * (fp / tot_fp)
            else:
                auc += (cov_tp / tot_tp) * (fp / tot_fp) + (tp / tot_tp) * (fp / tot_fp) / 2
                cov_tp += tp

        return auc

    # Calculate binary AUC for each class and store the results
    auc_results = {class_name: binary_auc(class_name) for class_name in df.columns}

    # Calculate weighted AUC
    class_counts = pd.Series(correctlabels).value_counts(normalize=True)
    
    weighted_auc = sum(auc_results[class_name] * class_counts.get(class_name, 0) for class_name in df.columns)

    return weighted_auc

# BRIER_SCORE

def brier_score(df, correctlabels):
    total_brier_score = 0.0

    for i in range(len(df)):
        pred = df.iloc[i].values  
        corr_label = correctlabels[i] 
        
        # Find the index of the correct label in the columns of the dataframe
        corr_label_i = np.where(df.columns == corr_label)[0][0]
        corr_vector = np.zeros(len(pred))
        corr_vector[corr_label_i] = 1
        squared_error = np.sum((pred - corr_vector) ** 2)
        total_brier_score += squared_error
    
    avg_brier_score = total_brier_score / len(df)
    
    return avg_brier_score

## 1. Define the class RandomForest

In [4]:
# Define the class RandomForest with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self - the object itself
#
# Output from __init__:
# <nothing>
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# column_filter, imputation, one_hot, labels, model
#

class RandomForest:
    def __init__(self):
        self.column_filter = None
        self.imputation = None
        self.one_hot = None
        self.labels = None
        self.model = None

# Input to fit:
# self      - the object itself
# df        - a dataframe (where the column names "CLASS" and "ID" have special meaning)
# no_trees  - no. of trees in the random forest (default = 100)
#
# Output from fit:
# <nothing>
#
# The result of applying this function should be:
#
# self.column_filter - a column filter (see Assignment 1) from df
# self.imputation    - an imputation mapping (see Assignment 1) from df
# self.one_hot       - a one-hot mapping (see Assignment 1) from df
# self.labels        - a (sorted) list of the categories of the "CLASS" column of df
# self.model         - a random forest, consisting of no_trees trees, where each tree is generated from a bootstrap sample
#                      and the number of evaluated features is log2|F| where |F| is the total number of features
#                      (for details, see lecture slides)
#
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Hint 1: First create the column filter, imputation and one-hot mappings
#
# Hint 2: Then get the class labels and the numerical values (as an ndarray) from the dataframe after dropping the class labels 
#
# Hint 3: Generate no_trees classification trees, where each tree is generated using DecisionTreeClassifier 
#         from a bootstrap sample (see lecture slides), e.g., generated by np.random.choice (with replacement) 
#         from the row numbers of the ndarray, and where a random sample of the features are evaluated in
#         each node of each tree, of size log2(|F|), where |F| is the total number of features;
#         see the parameter max_features of DecisionTreeClassifier
#

    def fit(self, df, no_trees=100):
        copy_df = df.copy()

        # Create all necessary mappings and store them in the self object
        copy_df, self.column_filter = create_column_filter(copy_df)
        copy_df, self.imputation = create_imputation(copy_df)
        copy_df, self.one_hot = create_one_hot(copy_df)

        # Get the unique labels from "CLASS" column and sort them
        self.labels = sorted(copy_df["CLASS"].astype("category").cat.categories)


        # Drop CLASS column and turn into a numpy array
        features = copy_df.drop(columns=["CLASS"]).to_numpy()

        random_forest = []
        # Create a random forest with size of no_trees
        for i in range(no_trees):
            # Create a bootstrap sample using np.random.choice to randomly replace some values with row numbers of the ndarray
            bootstrap_sample = np.random.choice(len(features), len(features), replace=True)

            # Generate a decision tree using the DecisionTreeClassifier and the number of evaluated features is log2|F|
            decision_tree = DecisionTreeClassifier(max_features="log2")
            
            # Fit the decision tree using the bootstrap sample and the labels
            decision_tree.fit(features[bootstrap_sample], copy_df['CLASS'].to_numpy()[bootstrap_sample])

            # Add each decision tree to the random forest
            random_forest.append(decision_tree)

        # Add the decision tree to the model which will end up as a RandomForest
        self.model = random_forest

# Input to predict:
# self - the object itself
# df   - a dataframe
# 
# Output from predict:
# predictions - a dataframe with class labels as column names and the rows corresponding to
#               predictions with estimated class probabilities for each row in df, where the class probabilities
#               are the averaged probabilities output by each decision tree in the forest
#
# Hint 1: Drop any "CLASS" and "ID" columns of the dataframe first and then apply column filter, imputation and one_hot
#
# Hint 2: Iterate over the trees in the forest to get the prediction of each tree by the method predict_proba(X) where 
#         X are the (numerical) values of the transformed dataframe; you may get the average predictions of all trees,
#         by first creating a zero-matrix with one row for each test instance and one column for each class label, 
#         to which you add the prediction of each tree on each iteration, and then finally divide the prediction matrix
#         by the number of trees.
#
# Hint 3: You may assume that each bootstrap sample that was used to generate each tree has included all possible
#         class labels and hence the prediction of each tree will contain probabilities for all class labels
#         (in the same order). Note that this assumption may be violated, and this limitation will be addressed 
#         in the next part of the assignment. 

    def predict(self, df):
        copy_df = df.copy()

        # Drop clumn "CLASS" and "ID" from the df
        copy_df.drop(columns=["CLASS"], inplace=True)

        # Apply all necessary mapping that were created in the fit method to the df
        copy_df = apply_column_filter(copy_df, self.column_filter)
        copy_df = apply_imputation(copy_df, self.imputation)
        copy_df = apply_one_hot(copy_df, self.one_hot)

        # Create a matrix filled with zeros contatining one row for each test instance and one column for each class label
        prediction_matrix = np.zeros((len(copy_df), len(self.labels)))

        for tree in self.model:
            # Get the prediction of the values for each tree in the RandomForest
            prediction = tree.predict_proba(copy_df.values)

            # Add the prediction of each tree on each iteration which will end up as a matrix with all total predictions for each tree
            prediction_matrix += prediction
            
        # Get the average preiction of all trees
        prediction_matrix = prediction_matrix / len(self.model)

        # Create a dataframe with the average predictions of all trees from the prediction matrix
        predictions = pd.DataFrame(prediction_matrix, columns=self.labels)

        return predictions


In [5]:
# Test your code (leave this part unchanged, except for if auc is undefined)

train_df = pd.read_csv("tic-tac-toe_train.csv")

test_df = pd.read_csv("tic-tac-toe_test.csv")

rf = RandomForest()

t0 = time.perf_counter()
rf.fit(train_df)
print("Training time: {:.2f} s.".format(time.perf_counter()-t0))

test_labels = test_df["CLASS"]

t0 = time.perf_counter()
predictions = rf.predict(test_df)

print("Testing time: {:.2f} s.".format(time.perf_counter()-t0))

print("Accuracy: {:.4f}".format(accuracy(predictions,test_labels)))
print("AUC: {:.4f}".format(auc(predictions,test_labels))) # Comment this out if not implemented in assignment 1
print("Brier score: {:.4f}".format(brier_score(predictions,test_labels))) # Comment this out if not implemented in assignment 1

Training time: 0.13 s.
Testing time: 0.05 s.
Accuracy: 0.5616
AUC: 0.5962
Brier score: 0.4935


In [6]:
train_labels = train_df["CLASS"]
predictions = rf.predict(train_df)
print("Accuracy on training set: {0:.4f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.4f}".format(auc(predictions,train_labels))) # Comment this out if not implemented in assignment 1
print("Brier score on training set: {0:.4f}".format(brier_score(predictions,train_labels))) # Comment this out if not implemented in assignment 1

Accuracy on training set: 0.6117
AUC on training set: 0.5989
Brier score on training set: 0.4712


### Comment on assumptions, things that do not work properly, etc.


## 2a. Handling trees with non-aligned predictions

In [6]:
# Define a revised version of the class RandomForest with the same input and output as described in part 1 above,
# where the predict function is able to handle the case where the individual trees are trained on bootstrap samples
# that do not include all class labels in the original training set. This leads to that the class probabilities output
# by the individual trees in the forest do not refer to the same set of class labels.
#
# Hint 1: The categories obtained with <pandas series>.cat.categories are sorted in the same way as the class labels
#         of a DecisionTreeClassifier; the latter are obtained by <DecisionTreeClassifier>.classes_ 
#         The problem is that classes_ may not include all possible labels, and hence the individual predictions 
#         obtained by <DecisionTreeClassifier>.predict_proba may be of different length or even if they are of the same
#         length do not necessarily refer to the same class labels. You may assume that each class label that is not included
#         in a bootstrap sample should be assigned zero probability by the tree generated from the bootstrap sample. 
#
# Hint 2: Create a mapping from the complete (and sorted) set of class labels l0, ..., lk-1 to a set of indexes 0, ..., k-1,
#         where k is the number of classes
#
# Hint 3: For each tree t in the forest, create a (zero) matrix with one row per test instance and one column per class label,
#         to which one column is added at a time from the output of t.predict_proba 
#
# Hint 4: For each column output by t.predict_proba, its index i may be used to obtain its label by t.classes_[i];
#         you may then obtain the index of this label in the ordered list of all possible labels from the above mapping (hint 2); 
#         this index points to which column in the prediction matrix the output column should be added to 



In [7]:
# Test your code (leave this part unchanged, except for if auc is undefined)

train_df = pd.read_csv("anneal_train.csv")

test_df = pd.read_csv("anneal_test.csv")

rf = RandomForest()

t0 = time.perf_counter()
rf.fit(train_df)
print("Training time: {:.2f} s.".format(time.perf_counter()-t0))

test_labels = test_df["CLASS"]

t0 = time.perf_counter()
predictions = rf.predict(test_df)
print("Testing time: {:.2f} s.".format(time.perf_counter()-t0))

print("Accuracy: {:.4f}".format(accuracy(predictions,test_labels)))
print("AUC: {:.4f}".format(auc(predictions,test_labels))) # Comment this out if not implemented in assignment 1
print("Brier score: {:.4f}".format(brier_score(predictions,test_labels))) # Comment this out if not implemented in assignment 1

Training time: 0.15 s.
Testing time: 0.06 s.
Accuracy: 0.9488
AUC: 0.9741
Brier score: 0.0975


## 2b. Estimate predictive performance using out-of-bag predictions

In [22]:
# Define an extended version of the class RandomForest with the same input and output as described in part 2a above,
# where the results of the fit function also should include:
# self.oob_acc - the accuracy estimated on the out-of-bag predictions, i.e., the fraction of training instances for 
#                which the given (correct) label is the same as the predicted label when using only trees for which
#                the instance is out-of-bag
#
# Hint 1: You may first create a zero matrix with one row for each training instance and one column for each class label
#         and one zero vector to allow for storing aggregated out-of-bag predictions and the number of out-of-bag predictions
#         for each training instance, respectively. By "aggregated out-of-bag predictions" is here meant the sum of all 
#         predicted probabilities (one sum per class and instance). These sums should be divided by the number of predictions
#         (stored in the vector) in order to obtain a single class probability distribution per training instance. 
#         This distribution is considered to be the out-of-bag prediction for each instance, and e.g., the class that 
#         receives the highest probability for each instance can be compared to the correct label of the instance, 
#         when calculating the accuracy using the out-of-bag predictions.
#
# Hint 2: After generating a tree in the forest, iterate over the indexes that were not included in the bootstrap sample
#         and add a prediction of the tree to the out-of-bag prediction matrix and update the count vector
#
# Hint 3: Note that the input to predict_proba has to be a matrix; from a single vector (row) x, a matrix with one row
#         can be obtained by x[None,:]
#
# Hint 4: Finally, divide each row in the out-of-bag prediction matrix with the corresponding element of the count vector
#
#         For example, assuming that we have two class labels, then we may end up with the following matrix:
#
#         2 4
#         4 4
#         5 0
#         ...
#
#         and the vector (no. of predictions) (6, 8, 5, ...)
#
#         The resulting class probability distributions are:
#
#         0.333... 0.666...
#         0.5 0.5
#         1.0 0

class RandomForest:

    def __init__(self):
        self.column_filter = None
        self.imputation = None
        self.one_hot = None
        self.labels = None
        self.model = None
        self.label_mapping = None
        self.oob_acc = None

    def fit(self, input_df, no_trees=100):
        # Same function as used in 1 and 2a but slightly modified
        df = input_df.copy()

        # Apply preprocessing steps and store the transformations
        df, self.column_filter = create_column_filter(df)
        df, self.imputation = create_imputation(df)
        df, self.one_hot = create_one_hot(df)
        
        # Extract class labels and create a mapping to numerical values
        self.labels = list(df["CLASS"].astype("category").cat.categories)
        self.label_mapping = dict(zip(self.labels, list(range(len(self.labels)))))

        # Extract features from the preprocesed df
        features = df.drop(columns=["ID", "CLASS"], errors='ignore').to_numpy()

        # Lists to store individual decision tree models and their corresponding missing sample indices
        models = []
        missing_tree_indices = []

        # Train multiple decision trees using bootstrap samples
        for i in range(no_trees):
            sampled_rows = np.random.choice(len(features), len(features), replace=True)
            data = features[sampled_rows]
            labels = df['CLASS'].to_numpy()[sampled_rows]
            
            tree = DecisionTreeClassifier(max_features='log2')
            tree.fit(data, labels)
            models.append(tree)

            # id indices of samples not used in the bootstrap sample
            missing = [x for x in range(len(features)) if x not in sampled_rows]
            missing_tree_indices.append(missing)

        # Save the trained models and calculate out-of-bag accuracy
        self.model = models
        self.oob_acc = self.calculate_oob_acc(missing_tree_indices, features, df)

    # Calculating out of bag accuracy
    def calculate_oob_acc(self, missing_tree_indices, features, df):
        # Hint 1: create a zero matrix 
        probabilities = np.zeros((len(features), len(self.labels)))
        counts = np.zeros(len(features))

        # Iterate over each decision tree and missing sample indices
        for i, curr_missing_indices in enumerate(missing_tree_indices):
            tree = self.model[i]

            # Hint 2: for each missing sample, predict class probabilities using the current tree
            for row_index in curr_missing_indices:
                current_train_data = features[row_index].reshape(1, -1)
                result = tree.predict_proba(current_train_data)

                # Update the probability matrix based on predicted probabilities
                for j, column_label in enumerate(tree.classes_):
                    label_index = self.label_mapping[column_label]
                    probabilities[row_index][label_index] += result[0][j]

                # Increment count for normalization
                counts[row_index] += 1

        # Hint 3 and hint 4: calculate accuracy matrix and overall accuracy
        accuracy_matrix = probabilities / counts[:, None]
        accuracy_df = pd.DataFrame(accuracy_matrix, columns=self.labels)
        result = accuracy(accuracy_df, df['CLASS'].astype("category"))

        return result

    def predict(self, input_df):
        df = input_df.copy()

        # Apply the same preprocessing steps used during training
        df = apply_column_filter(df, self.column_filter)
        df = apply_imputation(df, self.imputation)
        df = apply_one_hot(df, self.one_hot)
        df.drop(columns=["CLASS"], inplace=True)

        # Initialize matrix to store class probabilities for predictions
        probabilities = np.zeros((len(df), len(self.labels)))

        # Iterate over each decision tree and input samples for prediction
        for tree in self.model:
            for i, row in enumerate(df.values):

                # Get the classes of the current tree
                classes = tree.classes_
                # Predict class probabilities for the current sample
                result = tree.predict_proba(row.reshape(1, -1))

                # Update the probability matrix based on predicted probabilities
                for j, col in enumerate(classes):
                    label_i = self.label_mapping[col]
                    probabilities[i][label_i] += result[0][j]

        # Normalize probabilities
        probabilities /= len(self.model)
        predictions = pd.DataFrame(probabilities, columns=self.labels)

        return predictions

In [20]:
# Test your code (leave this part unchanged, except for if auc is undefined)

train_df = pd.read_csv("anneal_train.csv")

test_df = pd.read_csv("anneal_test.csv")

rf = RandomForest()

t0 = time.perf_counter()
rf.fit(train_df)
print("Training time: {:.2f} s.".format(time.perf_counter()-t0))

print("OOB accuracy: {:.4f}".format(rf.oob_acc))

test_labels = test_df["CLASS"]

t0 = time.perf_counter()
predictions = rf.predict(test_df)
print("Testing time: {:.2f} s.".format(time.perf_counter()-t0))

print("Accuracy: {:.4f}".format(accuracy(predictions,test_labels)))
print("AUC: {:.4f}".format(auc(predictions,test_labels))) # Comment this out if not implemented in assignment 1
print("Brier score: {:.4f}".format(brier_score(predictions,test_labels))) # Comment this out if not implemented in assignment 1

Training time: 2.58 s.
OOB accuracy: 0.9399
Testing time: 4.65 s.
Accuracy: 0.7016
AUC: 0.6963
Brier score: 0.4034


In [21]:
train_labels = train_df["CLASS"]
rf = RandomForest()
rf.fit(train_df)
predictions = rf.predict(train_df)
print("Accuracy on training set: {0:.2f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.2f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.2f}".format(brier_score(predictions,train_labels)))

Accuracy on training set: 0.67
AUC on training set: 0.71
Brier score on training set: 0.43


### Comment on assumptions, things that do not work properly, etc.